In [1]:
# Dependencies and Setpu
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API Key
from config import g_key

gmaps.configure(api_key=g_key)

In [2]:
# Path to the csv file
csv_path = "../WeatherPy/CSV Data/weather_city_data.csv"

# Changed the csv file to a DataFrame
city_weather_data = pd.read_csv(csv_path)
city_weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hofn,64.25,-15.21,35.62,74,16,5.79,IS,1603257243
1,Shingū,33.73,135.98,71.60,53,40,14.99,JP,1603257514
2,Airai,-8.93,125.41,80.98,38,12,3.87,TL,1603257294
3,Albany,42.60,-73.97,51.01,88,100,3.51,US,1603257251
4,Tomatlán,19.93,-105.25,77.70,90,83,0.69,MX,1603257515


In [3]:
# Used Lat and Lng as locations and Humidity as the weight
humidity = city_weather_data["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_weather_data[["Lat", "Lng"]]

# Configure gmaps and added heatmap layer to map
fig = gmaps.figure()
heat_humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=3)
fig.add_layer(heat_humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrowed the DataFrame to fit the weather conditions
narrow_down_df = city_weather_data.loc[(city_weather_data["Max Temp"] < 80) & (city_weather_data["Max Temp"] > 70) & (city_weather_data["Wind Speed"] < 10) & (city_weather_data["Cloudiness"] == 0), :]

# Drop any rows with null values
narrow_down_df = narrow_down_df.dropna(how="any")
narrow_down_df.reset_index(inplace=True)
del narrow_down_df['index']
narrow_down_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Najrān,17.49,44.13,75.20,17,0,0.43,SA,1603257279
1,Broken Hill,-31.95,141.43,78.80,19,0,8.05,AU,1603257469
2,Nampula,-15.12,39.27,71.60,78,0,6.93,MZ,1603257576
3,Álamos,27.02,-108.93,73.00,80,0,1.01,MX,1603257583
4,Sokoto,13.06,5.24,77.97,31,0,3.13,NG,1603257587


In [5]:
# Create empty list to store hotel names
hotel = []

# Run a for loop
for index, i in narrow_down_df.iterrows():
    lat = i["Lat"]
    lng = i["Lng"]
    
    # Set parameters to search for hotels within 5000 meters
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "lodging",
        "key": g_key
    }
    
    # create url, use the requests() and include the params, and use .json()
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    request = request.json()
    
    # Append the hotel names to the empty list
    try:
       
        narrow_down_df.loc[index, "Hotel Name"] = request['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Hotel name could not be found")
        
# Store the hotel list to the DataFrame and drop any null values         
# narrow_down_df["Hotel Name"] = hotel
# narrow_down_df = narrow_down_df.dropna(how="any")
narrow_down_df = narrow_down_df.dropna(how="any").reset_index(drop=True)
update_narrow_down_df = narrow_down_df.drop([0, 1])
update_narrow_down_df

Hotel name could not be found
Hotel name could not be found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,Nampula,-15.12,39.27,71.60,78,0,6.93,MZ,1603257576,Tacab Residencial
3,Álamos,27.02,-108.93,73.00,80,0,1.01,MX,1603257583,Luz del Sol
4,Sokoto,13.06,5.24,77.97,31,0,3.13,NG,1603257587,RG Palace
5,Ginda,30.69,78.49,75.20,57,0,4.92,IN,1603257588,Bal uma sankar hotel and restaurant
6,Iranshahr,27.20,60.68,77.00,6,0,1.83,IR,1603257615,کافی نت دانشگاه
7,Kalabo,-14.97,22.68,71.11,23,0,6.87,ZM,1603257617,Sikakubete Guest House Kalabo Town Council
8,Bilma,18.69,12.92,76.80,18,0,4.76,NE,1603257622,Expeditions Ténére voyages
9,Ampanihy,-24.70,44.75,78.71,40,0,3.53,MG,1603257623,Angora Hotel
10,Ābādān,30.34,48.30,73.40,53,0,4.70,IR,1603257639,Amir Kabir Hotel
11,Tahoua,14.89,5.27,77.00,61,0,8.97,NE,1603257650,Bungalows Guesthouse


In [6]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_down_df.iterrows()]
locations = narrow_down_df[["Lat", "Lng"]]

In [7]:
# Add marker layer on heap map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))